# Java

Java is the mother of all JVM languages, first released in 1995 after years of development by Sun Microsystems.  BeakerX uses [OpenJDK](http://openjdk.java.net/) for Java and all the kernels.

BeakerX supports Java cells that define classes, which can be used by other JVM based cells.
One cell is equivalent to a Java compilation unit and can contain a single class, or a sequence of statements.

In [1]:
%classpath add mvn org.json json 20180813

In [2]:
import org.json.JSONObject;
import org.json.JSONArray;

import java.io.FileReader;
import java.io.BufferedReader;

import java.io.FileNotFoundException;
import java.io.IOException;

import java.io.InputStreamReader;
import java.net.URL;
import java.net.URLEncoder;
import java.util.Collection;
import java.util.List;
import java.util.ArrayList;
import java.util.Scanner;

String adr="60, Rue Réaumur";//"28, rue notre dame des champs";
double score= -Double.MAX_VALUE;
double latitude= Double.NaN;
double longitude= Double.NaN;
        URL url= new URL("https://api-adresse.data.gouv.fr/search/?q="+URLEncoder.encode(adr, "UTF-8")+"&lat=48.8592112&lon=2.338452");
        try(BufferedReader br = new BufferedReader(new InputStreamReader(url.openStream()))) {
            JSONArray fs= new JSONObject(br.readLine()).getJSONArray("features");
            for(int i=0; i != fs.length(); ++i){
                JSONObject f= fs.getJSONObject(i);
               if(f.getJSONObject("properties").getString("city").equals("Paris")){
                   if(f.getJSONObject("properties").getDouble("score")> score){
                       score= f.getJSONObject("properties").getDouble("score");
                    longitude=f.getJSONObject("geometry").getJSONArray("coordinates").getDouble(0);   
                   latitude=f.getJSONObject("geometry").getJSONArray("coordinates").getDouble(1);   
                   }
                }
                
            }
System.out.println("latitude: "+latitude+", longitude: "+longitude);
            //JSONArray res = new JSONObject(resp).getJSONArray("features").getJSONObject(0).getJSONObject("geometry").getJSONArray("coordinates");
    //System.out.println(res);//longitude , latitude
        }


latitude: 48.867413, longitude: 2.347572


null

In [3]:
package test.beaker;

import java.util.Date;
import java.text.SimpleDateFormat;

public class BeakerTest {
  private Date _date;
  private SimpleDateFormat sdf = new SimpleDateFormat("yyyy-MM-dd'T'HH:mmZ");
  
  public BeakerTest() {
    _date = new Date();
  }

  public String getDateTxt() {
    return "Today:" + sdf.format(_date);
  }

  public String getDateUpperCaseTxt() {
    return getDateTxt().toUpperCase();
  }

}


test.beaker.BeakerTest

In [4]:
import java.io.FileReader;
import java.io.BufferedReader;

import java.io.FileNotFoundException;
import java.io.IOException;

import java.io.InputStreamReader;
import java.net.URL;
import java.net.URLEncoder;

import org.json.JSONObject;
import org.json.JSONArray;

import java.util.Collection;
import java.util.List;
import java.util.ArrayList;
import java.util.Map;
import java.util.HashMap;
import java.util.Scanner;

public class GeoPoint{
    public final double latitude;
    public final double longitude;
    public final Map<GeoPoint, Integer> neighborToDistance;
    
    public static List<GeoPoint> data= new ArrayList<GeoPoint>();
    
    public GeoPoint(double latitude, double longitude){
        this.latitude= latitude;
        this.longitude= longitude;
	neighborToDistance= new HashMap<GeoPoint, Integer>();
    }

    public double distanceTo(GeoPoint other){
        return distanceTo(other.latitude, other.longitude);
    }
   public double distanceTo(double latitude, double longitude){
       double earthRadius= 6371e3;// in meters
        double phi1=Math.toRadians(this.latitude);
        double phi2=Math.toRadians(latitude);
        double deltaPhi=Math.toRadians(latitude- this.latitude);
        double deltaLambda= Math.toRadians(longitude- this.longitude);
        double a= Math.sin(deltaPhi/2) * Math.sin(deltaPhi/2) +
            Math.cos(phi1) * Math.cos(phi2) * Math.sin(deltaLambda/2) * Math.sin(deltaLambda/2);
        double c= 2* Math.atan2(Math.sqrt(a), Math.sqrt(1-a));
        return earthRadius * c;
    }
    public String toString(){
        return "GeoPoint: "+super.toString()+"{latitude: "+latitude+", longitude: " + longitude+", nb of neighbors"+neighborToDistance.size()+"}";
    }
    public static GeoPoint closerTo(double latitude, double longitude, double deltaD, Collection<GeoPoint> points){
        GeoPoint res=null;
        double minD= Double.POSITIVE_INFINITY;
        for(GeoPoint point : points){
            double currentD= point.distanceTo(latitude, longitude);
            if((currentD < minD) && (currentD > deltaD) ){
                minD= currentD;
                res= point;
            }
        }
        return res;
    }
    public static GeoPoint findByName(String name){
	    GeoPoint res= null;
        try{
            
            URL url= new URL("https://api-adresse.data.gouv.fr/search/?q="+URLEncoder.encode(name, "UTF-8")+"&postcode=75000");
            try(BufferedReader br = new BufferedReader(new InputStreamReader(url.openStream()))) {
                JSONArray coords = new JSONObject(br.readLine()).getJSONArray("features").getJSONObject(0).getJSONObject("geometry").getJSONArray("coordinates");
                res= new GeoPoint(coords.getDouble(0), coords.getDouble(1));
            }catch(Exception e){
                System.err.println(e);
            }
        }catch(Exception e){
            System.err.println(e);
        }  
    return res; // should throw
    }

    public static GeoPoint closerTo(String name, Collection<GeoPoint> data){
        return closerTo(findByName(name), data);
    }
    public static GeoPoint closerTo(GeoPoint ref, Collection<GeoPoint> data){
        return closerTo(ref.latitude, ref.longitude, 1.e-50, data);
    }
public static List<GeoPoint> read(String dataUrl) throws IOException {
        List<GeoPoint> res= new ArrayList<GeoPoint>();
        URL url= new URL(dataUrl);
        try(BufferedReader br = new BufferedReader(new InputStreamReader(url.openStream()))) {
            String[] headers= br.readLine().split(" ");
            int nbVertices= Integer.parseInt(headers[0]);
            int nbEdges= Integer.parseInt(headers[1]);
            for(int i=0; i != nbVertices; ++i){
                String[] coords= br.readLine().split(" ");//latitude, longitude
                res.add(new GeoPoint(Double.parseDouble(coords[0]), Double.parseDouble(coords[1])));
            }
            for(int i=0; i != nbEdges; ++i){
                String[] data= br.readLine().split(" ");//start end nbDirs Duration Length
                GeoPoint start= res.get(Integer.parseInt(data[0]));
                GeoPoint end= res.get(Integer.parseInt(data[1]));
                Integer length= Integer.parseInt(data[4]);
                start.neighborToDistance.put(end, length);
                if(data[2].equals("2")){
                    end.neighborToDistance.put(start, length);
                }
            }
        }catch (Exception e){
            System.err.println(e);
        }
        return res;
    }
    public static void main(String[] args){
        try{
            String url="https://raw.githubusercontent.com/jilljenn/tryalgo/master/examples/paris.txt";
            data= read(url);
        }catch(FileNotFoundException e){
            System.err.println(e);
        }
        catch(IOException e){
            System.err.println(e);
        }
    }
    
}

com.twosigma.beaker.javash.bkr4366a878.GeoPoint

In [5]:
String[] args={};
GeoPoint.main(args);

null

In [5]:
double minLat= Double.MAX_VALUE;
double maxLat= -Double.MAX_VALUE;
double minLong= Double.MAX_VALUE;
double maxLong= -Double.MAX_VALUE;
double sumLat= 0.;
double sumLong= 0.;
for(GeoPoint p: GeoPoint.data){
    sumLat+= p.latitude;
    sumLong+= p.longitude;
    if(p.latitude < minLat){ minLat= p.latitude;}
    if(p.latitude > maxLat){ maxLat= p.latitude;}
    if(p.longitude < minLong){ minLong= p.longitude;}
    if(p.latitude > maxLong){ maxLong= p.longitude;}
}
System.out.println("lat:["+minLat+", "+maxLat+"], long:["+minLong+", "+maxLong+"]");
System.out.println("avg lat: "+(sumLat/GeoPoint.data.size())+", avg long: "+(sumLong/GeoPoint.data.size()));

lat:[48.8157828, 48.901557700000005], long:[2.2519666000000003, 2.3598348000000002]
avg lat: 48.859211258010134, avg long: 2.338452206106807


null

In [ ]:
import java.util.Arrays;
import java.util.List;
import java.util.ArrayList;

import com.twosigma.beakerx.chart.xychart.plotitem.*;
Plot map = new Plot();
Rasters r= new Rasters();
r.setFilePath("Paris.png");
r.setOpacity(Arrays.asList(new Number[]{.5}));
r.setX(Arrays.asList(new Object[]{2.2519666000000003}));
r.setY(Arrays.asList(new Number[]{48.901557700000005}));
r.setHeight(Arrays.asList(new Number[]{48.901557700000005-48.8157828}));
r.setWidth(Arrays.asList(new Number[]{2.415388-2.2519666000000003}));
map.add(r);
for(GeoPoint src : GeoPoint.data){
    for(GeoPoint dest: src.neighbors.keySet()){
        Line way= new Line();
        way.setX(new Object[]{src.longitude, dest.longitude});
        way.setY(new Number[]{src.latitude, dest.latitude});
        if(!dest.neighbors.keySet().contains(src)){
            way.setColor(Color.red);
        }
        map.add(way);
}
return map;

In [13]:
import java.util.Arrays;
import java.util.List;
import java.util.ArrayList;

import com.twosigma.beakerx.chart.xychart.plotitem.*;
Plot map = new Plot();
Rasters r= new Rasters();
r.setFilePath("Paris.png");
r.setOpacity(Arrays.asList(new Number[]{.5}));
r.setX(Arrays.asList(new Object[]{2.2519666000000003}));
r.setY(Arrays.asList(new Number[]{48.901557700000005}));
r.setHeight(Arrays.asList(new Number[]{48.901557700000005-48.8157828}));
r.setWidth(Arrays.asList(new Number[]{2.415388-2.2519666000000003}));
map.add(r);

Points p= new Points();
ArrayList<Object> pXs= new ArrayList<Object>();
ArrayList<Number> pYs= new ArrayList<Number>();
for(GeoPoint pt : GeoPoint.data){
    pXs.add(pt.longitude);
    pYs.add(pt.latitude);
}
p.setX(pXs);
p.setY(pYs);
p.setShape(ShapeType.LINECROSS);
p.setSize(Arrays.asList(new Number[]{1}));
map.add(p);
return map;

In [13]:
import java.util.Arrays;
import java.util.List;
String test="test";
Plot map = new Plot();
Rasters r= new Rasters();
r.setFilePath("Paris.png");
Object[] xs={-132};
    Number[] ys={56};
        List<Object> x=Arrays.asList(xs);
        List<Number> y=Arrays.asList(ys);
r.setX(x);
r.setY(y);
Number[] hs={32};
List<Number> h= Arrays.asList(hs);
r.setHeight(h);
Number[] ws={66};
List<Number> w= Arrays.asList(ws);
r.setWidth(w);
map.add(r);
return map;

In [7]:
package test.beaker;

BeakerTest bt = new BeakerTest();
return bt.getDateTxt();

Today:2018-08-26T06:58+0000

In [ ]:
import java.util.List;
import java.util.ArrayList;
import com.twosigma.beakerx.chart.xychart.Plot;
import com.twosigma.beakerx.chart.xychart.plotitem.*;
import com.twosigma.beakerx.chart.Color;

Plot p = new Plot();

p.setTitle("this is a Java plot");

Bars b = new Bars();

List<Number> yList = new ArrayList<Number>();
yList.add(2);
yList.add(5);
yList.add(4);
yList.add(8);

b.setY(yList);
b.setColor(Color.blue);
b.setWidth(0.5);

p.add(b);
  
return p;

In [ ]:
package test.beaker;
interface DateGetter {
   public String getDateTxt();
}

In [ ]:
package test.beaker;
public class DG2 extends BeakerTest implements DateGetter {
}

## Using the classpath to load a jar

In [ ]:
%classpath add jar ../resources/jar/BeakerXClasspathTest.jar

In [ ]:
System.out.println("The Groovy working folder is :");
System.out.println(java.nio.file.Paths.get(".").toAbsolutePath().normalize().toString());
System.out.println("BeakerXClasspathTest.jar exists in this folder");

In [ ]:
import com.beaker.BeakerXClasspathTest;

BeakerXClasspathTest t = new BeakerXClasspathTest();
System.out.println(com.beaker.BeakerXClasspathTest.staticTest);
System.out.println(t.getObjectTest());